# MACHINE LEARNING — ASSIGNMENT 2

**Topic:** Multi-model classification + Streamlit deployment  
**Planned Dataset:** Credit Card Default (UCI/Kaggle)

---


## STUDENT INFORMATION


- **Name:** HARISH BHATT  
- **StudentID:** 2025AA05578@wilp.bits-pilani.ac.in
- **Program:** M.Tech (AIML/DSE)  
- **Course:** Machine Learning  
- **Assignment:** 2  
- **Submission Date:** 15-Feb-2026  


## 1) Imports & Environment Check

In [ ]:
# If running in Colab, uncomment and install dependencies as needed
!pip -q install pandas numpy scikit-learn xgboost matplotlib seaborn joblib

import os
import sys
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, roc_auc_score,
    precision_score, recall_score,
    f1_score, matthews_corrcoef,
    confusion_matrix, classification_report
)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier

import joblib

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("Python:", sys.version.split()[0])
print("pandas:", pd.__version__)


## 2) Load Dataset

In [ ]:
# Load pre-split training and testing datasets directly from GitHub

train_url = "https://raw.githubusercontent.com/Harishbhatt07/ML-Assignment-2/main/model/data/train.csv"
test_url = "https://raw.githubusercontent.com/Harishbhatt07/ML-Assignment-2/main/model/data/test.csv"

train_df = pd.read_csv(train_url)
test_df = pd.read_csv(test_url)

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

TARGET_COL = "default.payment.next.month"

X_train = train_df.drop(columns=[TARGET_COL])
y_train = train_df[TARGET_COL]

X_test = test_df.drop(columns=[TARGET_COL])
y_test = test_df[TARGET_COL]

In [ ]:
import pandas as pd

# Sanity checks (REQUIRED)
MIN_ROWS = 500
MIN_FEATURES = 12

print("Columns:", len(train_df.columns))
print("Rows   :", len(train_df))

# Basic shape checks per assignment requirement
assert len(train_df) >= MIN_ROWS, f"Dataset has {len(train_df)} rows; need at least {MIN_ROWS}."
# features check will be done after defining X (excluding target)

## 3) Quick EDA / Data Checks (Optional but useful)

In [ ]:
# Basic info
display(train_df.describe(include='all').T.head(30))
print("\nMissing values per column (top 20):")
display(train_df.isna().sum().sort_values(ascending=False).head(20))

# Class balance (update target column name if needed)
TARGET_COL = "default.payment.next.month"
if TARGET_COL in train_df.columns:
    display(train_df[TARGET_COL].value_counts(dropna=False))
else:
    print(f"Target column '{TARGET_COL}' not found. Columns are:", list(train_df.columns))

## 4) Preprocessing

In [ ]:
# -----------------------------
# Preprocessing for Train/Test
# -----------------------------

TARGET_COL = "default.payment.next.month"  # confirm exact column name

# 4.1 Drop ID column if present (Train)
if "ID" in train_df.columns:
    train_df = train_df.drop(columns=["ID"])

# 4.2 Drop ID column if present (Test)
if "ID" in test_df.columns:
    test_df = test_df.drop(columns=["ID"])

# 4.3 Define X and y separately
X_train = train_df.drop(columns=[TARGET_COL])
y_train = train_df[TARGET_COL]

X_test = test_df.drop(columns=[TARGET_COL])
y_test = test_df[TARGET_COL]

print("Train X shape:", X_train.shape)
print("Test X shape:", X_test.shape)

In [ ]:
# Feature/target sanity checks (REQUIRED)

MIN_FEATURES = 12

# 1) Check target exists in both train and test
assert TARGET_COL in train_df.columns, f"Target column not found in train_df: {TARGET_COL}"
assert TARGET_COL in test_df.columns, f"Target column not found in test_df: {TARGET_COL}"

# 2) Check feature count (on train)
n_features = X_train.shape[1]
print("Number of features (X_train):", n_features)
assert n_features >= MIN_FEATURES, f"Dataset has {n_features} features; need at least {MIN_FEATURES}."

# 3) Check train/test have same feature columns (to avoid Streamlit + model issues)
assert list(X_train.columns) == list(X_test.columns), "Train and test feature columns do not match!"

# 4) Target distribution (useful for observations)
print("\nTrain target distribution:")
display(y_train.value_counts(normalize=True))

print("\nTest target distribution:")
display(y_test.value_counts(normalize=True))

## 5) Scaling (for Logistic Regression + KNN; optional for others)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Scaled shapes:", X_train_scaled.shape, X_test_scaled.shape)

## 6) Evaluation Utilities

In [ ]:
def get_binary_proba(model, X):
    """Return P(class=1) for binary classifiers."""
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(X)[:, 1]
    elif hasattr(model, "decision_function"):
        # Convert decision scores to pseudo-probabilities via min-max scaling as fallback
        scores = model.decision_function(X)
        proba = (scores - scores.min()) / (scores.max() - scores.min() + 1e-12)
    else:
        raise ValueError("Model has neither predict_proba nor decision_function.")
    return proba


def evaluate_model(model, Xtr, Xte, ytr, yte, label_name=None):
    """Fit, predict, compute required metrics for binary classification."""
    model.fit(Xtr, ytr)
    y_pred = model.predict(Xte)
    y_prob = get_binary_proba(model, Xte)

    metrics = {
        "Accuracy": accuracy_score(yte, y_pred),
        "AUC": roc_auc_score(yte, y_prob),
        "Precision": precision_score(yte, y_pred, zero_division=0),
        "Recall": recall_score(yte, y_pred, zero_division=0),
        "F1": f1_score(yte, y_pred, zero_division=0),
        "MCC": matthews_corrcoef(yte, y_pred),
    }

    # Confusion matrix + report
    cm = confusion_matrix(yte, y_pred)
    report = classification_report(yte, y_pred, digits=4, zero_division=0)

    return metrics, cm, report

## 7) Train the 6 Required Models

In [ ]:
  # =========================
# Train + Evaluate 6 Models
# =========================

results = []
artifacts = {}  # store models + cm/report for later Streamlit use

# 1) Logistic Regression (scaled) — optional: class_weight="balanced"
lr = LogisticRegression(max_iter=2000, random_state=RANDOM_STATE)
m, cm, report = evaluate_model(lr, X_train_scaled, X_test_scaled, y_train, y_test)
results.append(("Logistic Regression", m))
artifacts["Logistic Regression"] = {"model": lr, "cm": cm, "report": report}

# 2) Decision Tree (unscaled)
dt = DecisionTreeClassifier(random_state=RANDOM_STATE)
m, cm, report = evaluate_model(dt, X_train, X_test, y_train, y_test)
results.append(("Decision Tree", m))
artifacts["Decision Tree"] = {"model": dt, "cm": cm, "report": report}

# 3) KNN (scaled)
knn = KNeighborsClassifier(n_neighbors=5)
m, cm, report = evaluate_model(knn, X_train_scaled, X_test_scaled, y_train, y_test)
results.append(("KNN", m))
artifacts["KNN"] = {"model": knn, "cm": cm, "report": report}

# 4) Naive Bayes (Gaussian) (scaled)
nb = GaussianNB()
m, cm, report = evaluate_model(nb, X_train_scaled, X_test_scaled, y_train, y_test)
results.append(("Naive Bayes", m))
artifacts["Naive Bayes"] = {"model": nb, "cm": cm, "report": report}

# 5) Random Forest (unscaled)
rf = RandomForestClassifier(
    n_estimators=150,
    max_depth=12,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
m, cm, report = evaluate_model(rf, X_train, X_test, y_train, y_test)
results.append(("Random Forest (Ensemble)", m))
artifacts["Random Forest (Ensemble)"] = {"model": rf, "cm": cm, "report": report}

# 6) XGBoost (unscaled)
xgb = XGBClassifier(
    n_estimators=400,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    random_state=RANDOM_STATE,
    eval_metric="logloss",
    n_jobs=-1
)
m, cm, report = evaluate_model(xgb, X_train, X_test, y_train, y_test)
results.append(("XGBoost (Ensemble)", m))
artifacts["XGBoost (Ensemble)"] = {"model": xgb, "cm": cm, "report": report}

# Comparison table
results_df = pd.DataFrame([r[1] for r in results], index=[r[0] for r in results])
display(results_df)

In [ ]:
# Results validation (REQUIRED — prevent accidental 0.0 / missing metrics)

REQUIRED_MODELS = [
    "Logistic Regression",
    "Decision Tree",
    "KNN",
    "Naive Bayes",
    "Random Forest (Ensemble)",
    "XGBoost (Ensemble)"
]

# Check all required models present
missing = [m for m in REQUIRED_MODELS if m not in results_df.index]
assert not missing, f"Missing model results for: {missing}"

# Check for NaNs
assert not results_df.isna().any().any(), "Some metrics are NaN. Check AUC/probabilities."

# Check metrics not all zeros
for col in results_df.columns:
    assert (results_df[col] != 0).any(), f"Column {col} looks all zeros — did metrics compute correctly?"

print("✅ Results table contains all required models and valid metric values.")
display(results_df)

## 8) Plots and Visualization

In [ ]:
# Bar Plot — Accuracy Comparison
plt.figure(figsize=(8,5))
results_df["Accuracy"].sort_values().plot(kind="barh")
plt.title("Accuracy Comparison Across Models")
plt.xlabel("Accuracy")
plt.show()

In [ ]:
#Bar Plot — AUC Comparison
plt.figure(figsize=(8,5))
results_df["AUC"].sort_values().plot(kind="barh")
plt.title("AUC Score Comparison Across Models")
plt.xlabel("AUC Score")
plt.show()

In [ ]:
#Grouped Bar Chart — Precision, Recall, F1 Comparison
metrics_to_plot = ["Precision", "Recall", "F1"]

results_df[metrics_to_plot].plot(kind="bar", figsize=(10,6))
plt.title("Precision, Recall, and F1 Comparison")
plt.ylabel("Score")
plt.xticks(rotation=45)
plt.legend(loc="lower right")
plt.show()

In [ ]:
# MCC Comparison Plot (Advanced Metric)
plt.figure(figsize=(8,5))
results_df["MCC"].sort_values().plot(kind="barh")
plt.title("Matthews Correlation Coefficient Comparison")
plt.xlabel("MCC Score")
plt.show()

## 9) Identify the Best Model

In [ ]:
# You can choose based on AUC, MCC, or F1. Here we pick best by AUC.
best_model_name = results_df["AUC"].idxmax()
best_model = artifacts[best_model_name]["model"]

print("Best model by AUC:", best_model_name)

## 10) Visuals: Confusion Matrix & Report (for all models)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for model_name, obj in artifacts.items():

    cm = obj["cm"]
    report = obj["report"]

    print("="*60)
    print(f"Model: {model_name}")
    print("="*60)
    print("Classification Report:\n")
    print(report)

    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt="d", cbar=False)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix — {model_name}")
    plt.show()

## 11) Save Artifacts for Streamlit

In [ ]:
# =========================
# Save Artifacts for Streamlit
# =========================
import os, joblib

os.makedirs("model", exist_ok=True)

# Save scaler (needed for LR + KNN + Naive Bayes)
joblib.dump(scaler, "model/scaler.pkl")

# Save each model
for name, obj in artifacts.items():
    safe_name = (
        name.lower()
        .replace(" ", "_")
        .replace("(", "")
        .replace(")", "")
        .replace("/", "_")
    )
    joblib.dump(obj["model"], f"model/{safe_name}.pkl")

print("Saved models + scaler into ./model/")

## 12) Observations


## Comparison Table of Evaluation Metrics

| ML Model Name | Accuracy | AUC | Precision | Recall | F1 | MCC |
|---------------|----------|------|-----------|--------|------|------|
| Logistic Regression | 0.8077 | 0.7076 | 0.6868 | 0.2396 | 0.3553 | 0.3244 |
| Decision Tree | 0.7145 | 0.6075 | 0.3694 | 0.4115 | 0.3893 | 0.2042 |
| KNN | 0.7928 | 0.7014 | 0.5487 | 0.3564 | 0.4322 | 0.3233 |
| Naive Bayes | 0.7525 | 0.7249 | 0.4515 | 0.5539 | 0.4975 | 0.3386 |
| Random Forest (Ensemble) | 0.8168 |	0.7741 |	0.6597 |	0.3549 |	0.4615 |	0.3883 |
| XGBoost (Ensemble) | **0.8167** | **0.7774** | 0.6553 | 0.3610 | 0.4655 | **0.3896** |

---

## Observations on Model Performance

| ML Model Name | Observation about Model Performance |
|---------------|-------------------------------------|
| Logistic Regression | Achieved good overall accuracy and high precision but very low recall, indicating conservative prediction of default cases. It tends to miss many actual defaulters. |
| Decision Tree | Showed the weakest overall performance across most metrics, likely due to overfitting and limited generalization capability. |
| KNN | Delivered moderate performance with balanced precision and recall but lower discrimination ability compared to ensemble models. |
| Naive Bayes | Achieved the highest recall among all models, indicating better sensitivity in detecting defaulters, though with reduced precision. |
| Random Forest (Ensemble) | Demonstrated strong performance with high AUC and MCC, indicating improved stability and generalization compared to individual models. |
| XGBoost (Ensemble) | Achieved the best overall performance with highest AUC and MCC. It provides superior class separation and balanced predictive capability. |

---

## Final Model Selection

Based on the comparison of evaluation metrics, **XGBoost** is selected as the best-performing model due to its superior AUC and MCC scores, indicating better overall classification quality and balanced performance on imbalanced data.